In [1]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, HumanMessage
from langchain.tools.retriever import create_retriever_tool
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [2]:
llm = Ollama(model = "llama2")
loader = WebBaseLoader("https://www.healthline.com/nutrition/mediterranean-diet-meal-plan")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OllamaEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
os.environ['TAVILY_API_KEY'] = ""

In [3]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
    """
)
prompt_search_query = ChatPromptTemplate.from_messages([
  MessagesPlaceholder(variable_name = "chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation"),
])
prompt_get_answer = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\\n\\n{context}"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("user", "{input}"),
])

In [4]:
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)
conversation_chain = create_stuff_documents_chain(llm, prompt_get_answer)
conversation_retriever_chain = create_retrieval_chain(retriever_chain, conversation_chain)
search = TavilySearchResults()
retriever_tool = create_retriever_tool(retriever, "Healthline_search", "For any questions about Healthline, you must use this tool")
tools = [retriever_tool, search]
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent = agent, tools = tools, handle_parsing_errors = True)

In [ ]:
agent_executor.invoke({"input": "Describe a mediterranean diet"})

In [ ]:
response = retrieval_chain.invoke({"input": "Create a healthy diet plan"})
print(response["answer"])
chat_history = [HumanMessage(content = "How many steps were there in plan?"), AIMessage(content = "10")]
response = conversation_retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Condense the plan so that there are fewer steps",
})
print(response["answer"])